In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback , ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
from tensorflow.keras.optimizers import (
    SGD,          # Stochastic Gradient Descent
    RMSprop,      # Root Mean Square Propagation
    Adam,         # Adaptive Moment Estimation
    AdamW,        # Adam with Weight Decay
    Nadam,        # Nesterov-accelerated Adam
    Adagrad,      # Adaptive Gradient Algorithm
    Adadelta,     # Adaptive Delta
    Ftrl          # Follow-the-Regularized-Leader
)
import nbformat

2025-08-11 10:56:54.176549: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-11 10:56:54.211906: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-11 10:56:54.225410: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-11 10:56:54.229577: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-11 10:56:54.264263: I tensorflow/core/platform/cpu_feature_guar

In [4]:

my_file = "./pentilfuran.MDE"
df = pd.read_csv(
    my_file,
    sep=r"\s+",            # înlocuiește delim_whitespace
    comment='#',
    names=["Step", "T", "E_KS", "E_tot", "Vol", "P"]
)

# === Selectare rapidă a liniilor 1:901 pentru fiecare Step ===
df_data = (
    df.groupby("Step", group_keys=False)
      .apply(lambda g: g.iloc[1:901])
      .reset_index(drop=True)
)

print(f"Dimensiunea după filtrare: {len(df_data)} rânduri")

# === Extrage energia și scalează ===
scaler = MinMaxScaler()
energy_scaled = scaler.fit_transform(df_data[['E_tot']]).flatten()

data = pd.DataFrame({
    'Step': df_data.index,
    'Energy_scaled': energy_scaled
})


Dimensiunea după filtrare: 90000 rânduri


/tmp/ipykernel_155320/655334710.py:12: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.iloc[1:901])


In [ ]:

# === Funcție creare secvențe pentru LSTM ===
def create_sequences(data, sequence_length, out_steps):
    sequences, targets = [], []
    for i in range(len(data) - sequence_length):
        seq = data[i:i + sequence_length]
        target = data[i + sequence_length:i + sequence_length + out_steps]
        if len(target) == out_steps:  # ne asigurăm că targetul e complet
            sequences.append(seq)
            targets.append(target)
    return np.array(sequences), np.array(targets)

# Parametrii
sequence_length = 200
OUT_STEPS = 100

sequences, targets = create_sequences(
    data['Energy_scaled'].values,
    sequence_length,
    OUT_STEPS
)

print(f"Shape secvențe: {sequences.shape}")
print(f"Shape ținte: {targets.shape}")

Shape secvențe: (89701, 200)
Shape ținte: (89701, 100)
